### SocialAL Models
# Data simulation using best-fit parameters - multiple subjects
KLS 8.30.19; update 7.7.22; update 10.31.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.11.0 (v3.11.0:deaf509e8f, Oct 24 2022, 14:43:23) [Clang 13.0.0 (clang-1300.0.29.30)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.stats import gamma
import matplotlib.pyplot as plt
import itertools
import csv

## Pull in functions

In [3]:
run ../common_functions.ipynb

In [4]:
run ../baseline_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    
    beta = params
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Trial': trial, 'Stim_Sequence': np.ndarray.tolist(trial_sequence), 'Choice' : choices, 'Trustee_Response': responses, 'Beta': [beta] * tn *3 }    
    return data

In [8]:
#sim_data(15,2)

### Pull in best-fit parameters from participants

In [9]:
dt = pd.read_csv('../../../output/baseline_model_params.csv')
print(dt)

          id      beta      X.LLH
0   sub-2013  0.113988  60.996952
1   sub-2007  0.816956  58.224363
2   sub-2006  0.671496  60.996952
3   sub-2012  0.088588  62.383246
4   sub-2004  0.333262  62.383246
..       ...       ...        ...
58  sub-2009  0.129989  62.383246
59  sub-2037  0.623544  58.224363
60  sub-2023  0.150704  60.996952
61  sub-2022  0.184520  60.996952
62  sub-2036  0.171241  62.383246

[63 rows x 3 columns]


In [10]:
#file = open('../../../output/socialAL_cut.csv', 'r')
#dt2 = list(csv.reader(file))
#file.close()
#cut = [item for sublist in dt2 for item in sublist]
#dt = dt1[~dt1['id'].isin(cut)]


In [11]:
betas = dt['beta']
parent_sub = dt['id']
print(betas)

0     0.113988
1     0.816956
2     0.671496
3     0.088588
4     0.333262
        ...   
58    0.129989
59    0.623544
60    0.150704
61    0.184520
62    0.171241
Name: beta, Length: 63, dtype: float64


### Simulate data

In [12]:
data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Beta', 'Subject'])
for p in range(len(betas)):
    for q in range(1,11):
        dt = sim_data(15, betas[p])
        dt['Subject'] = parent_sub[p] + '_' + str(q) #[p + 1] * 45
        dt = pd.DataFrame(dt)
        data = pd.concat([data, dt])
    
data.to_csv(path_or_buf = '../../../output/simulation/part_params/sim_baseline_model_data.csv', index = False)

In [13]:
# df = data[data['Subject']==120]
# df